In [2]:
# Import necessary packages
import pandas as pd

In [3]:
# Load dataset
df = pd.read_csv('job_opportunities.csv', encoding='ISO-8859-1')

### Cleaning

In [6]:
# Drop last row
df.drop(df.loc[df['Company'].isnull()].index, axis='rows', inplace=True)

In [9]:
df.drop_duplicates(inplace=True)

In [30]:
df_salary_isnull = df.loc[df['Salary'].isnull()]

In [31]:
df.drop(df_salary_isnull.index, inplace=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2995 entries, 0 to 3196
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Company                     2995 non-null   object
 1   Job Title                   2995 non-null   object
 2   Location                    2995 non-null   object
 3   Job Type                    2995 non-null   object
 4   Experience level            2768 non-null   object
 5   Salary                      2824 non-null   object
 6   Requirment of the company   2995 non-null   object
 7   Facilities                  2995 non-null   object
dtypes: object(8)
memory usage: 210.6+ KB


In [32]:
df['Salary'] = df['Salary'].apply(lambda x: str.strip(x))

In [34]:
# Create new column that contains stars True or False
df['Salary_has_star'] = df['Salary'].str.contains('\*')

In [35]:
df.loc[df['Salary_has_star'] == True].shape

(2261, 9)

In [44]:
df['Salary'] = df['Salary'].apply(lambda x: str.replace(x, '+', ''))
df['Salary'] = df['Salary'].apply(lambda x: str.replace(x, '*', ''))
df['Salary'] = df['Salary'].apply(lambda x: str.replace(x, 'K', ''))
df['Salary'] = df['Salary'].apply(lambda x: str.strip(x))

In [46]:
df['Currency'] = df['Salary'].unique()

array(['48', '90', '108', '184', '39', '140', '40', '80', '35', '62',
       '159', '97', '92', '129', '115', '44', '73', '59', '106', '105',
       '55', '110', '63', '49', '180', '127', '60', '45', '135', '69',
       '176', '65', '51', '67', '104', '166', '131', '43', '81', '96',
       '56', '130', '50', '100', '52', '76', '172', '119', '125', '116',
       '77', '74', '70', '120', '189', '149', '30', '167', '236', '179',
       '200', '139', '113', '150', '158', '133', '78', '36', '122',
       'EUR 81', '190', '72', '136', '102', '31', '89', '156', '107',
       '199', '224', '142', 'GBP 35', '183', '164', 'EUR 130', '174',
       '84', '79', '103', '143', '175', '170', '141', '283', '151', '111',
       '148', '204', '202', '295', '68', '195', '145', '267', '109',
       '310', '153', '154', '86', '207', '85', '66', '315', '160', '144',
       '245', '126', '182', '157', '214', '173', 'EUR 80', 'EUR 36',
       'EUR 60', 'GBP 42', '112', '124', '185', '147', 'GBP 80', '146',
   

In [41]:
df['Salary'][0]

'48'